# 1. Định nghĩa bài toán
---

- Lunar Lander là một bài toán học tăng cường, cụ thể là 1 bài toán Quyết định Markov (MDP) rời rạc.
$$M=\{S,A,P,R,\gamma\}$$
- Trong đó:
##### Không gian trạng thái ($S$)
- Theo định nghĩa từ Gymnasium, không gian trạng thái rời rạc $S \subset R^8$ 
- Mỗi trạng thái $s\in S$ là vector gồm 8 giá trị:
$$s=[x,y,v_{x},v_{y},\theta,\omega,c_{l},c_{r}]$$
- Với:
	- $x,y:$ toạ độ của con tàu theo trục $x,y$
	- $v_{x},v_{y}:$ vận tốc của con tàu theo các phương $x,y$
	- $\theta:$ góc quay của con tàu
	- $\omega:$ vận tốc quay của con tàu
	- $c_{l},c_{r}:$ biến nhị phân thể hiện sự tiếp đất của hai chân $left,right$
##### Không gian hành động ($A$)
- Theo định nghĩa từ Gymnasium, không gian hành động $A$ rời rạc gồm các hành động:
	- $a=0:$ không làm gì
	- $a=1:$ bật động cơ chính
	- $a=2:$ bật động cơ bên trái
	- $a=3:$ bật động cơ bên phải
##### Xác suất chuyển trạng thái ($P$)
##### Hàm phần thưởng ($R$)
- Theo định nghĩa từ Gymnasium, phần thưởng nhận được sau khi chuyển từ trạng thái $s$ sang $s'$ là: $$R(s,s')=\begin{cases}
+10p \text{ với mỗi chân tiếp xúc} \\
-0.03p \text{ với mỗi động cơ hai bên bật} \\
-0.3p \text{ với động cơ chính bật} \\
-100p \text{ với hạ cánh không thành công} \\
+100p \text{ với hạ cánh thành công} \\
\text{càng tăng/giảm khi tàu càng xa/gần mặt đất} \\
\text{càng tăng/giảm khi tốc độ tàu càng nhanh/chậm} \\
\text{càng giảm khi góc nghiêng tàu càng lớn}
\end{cases}$$
##### Hệ số chiết khấu (discount factor: $\gamma$)

#### 2. Trạng thái kết thúc
- Tàu hạ cánh thành công:
	- Cả 2 chân đều chạm đất: $c_{l}=c_{r}=1$
	- Vận tốc rơi nhỏ và góc nghiêng nhỏ
	- Tổng $\text{reward} \geq 200$
- Tàu rơi hoặc lật:
	- Chạm đất với vận tốc lớn hoặc góc nghiêng lớn
	- Rơi ra khỏi màn hinh
	- Tổng $\text{reward} \leq -100$
- Vượt quá số bước tối đa:
	- Mỗi episode có số step tối đa là 1000
	- Nếu không hạ cánh hay rơi trong khoảng số bước này, episode sẽ bị huỷ
#### 3. Mục tiêu bài toán
- Tìm chính sách tối ưu $\pi^*:S\to A$ sao cho kỳ vọng của tổng phần thưởng được tối đa
$$\pi^*=argmax_{\pi}E_{\pi}\left[ \sum_{t=0}^{\infty}\gamma^t.R(s_{t},a_{t},s_{t+1}) \right]$$

# 2. Cài đặt tổng quan
---

## 2.1. Thư viện, các hàm xử lí chung và môi trường LunarLander

In [61]:
# !pip install gymnasium
# !pip install swig
# !pip install gymnasium[box2d]
# !pip install torch
# !pip install numpy
# !pip install matplotlib

In [62]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from collections import deque
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import random
import glob
from IPython.display import Video
from tqdm import tqdm

In [63]:
def plot_rewards(rewards, moving_average_window=50):
    # Calculate moving average
    moving_avg = np.convolve(rewards, np.ones(moving_average_window)/moving_average_window, mode='valid')
    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(rewards, label='Rewards', alpha=0.5)
    plt.plot(np.arange(moving_average_window - 1, len(rewards)), moving_avg, label='Moving Average', color='red')
    plt.title('Rewards and Moving Average')
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.legend()
    plt.grid()
    plt.show()

In [64]:
def plot_losses(losses, moving_average_window=50):
    # Calculate moving average
    moving_avg = np.convolve(losses, np.ones(moving_average_window)/moving_average_window, mode='valid')
    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(losses, label='Losses', alpha=0.5)
    plt.plot(np.arange(moving_average_window - 1, len(losses)), moving_avg, label='Moving Average', color='red')
    plt.title('Losses and Moving Average')
    plt.xlabel('Episode')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

In [65]:
def plot_fuel_usage(fuel_usage, moving_average_window=50):
    # Calculate moving average
    moving_avg = np.convolve(fuel_usage, np.ones(moving_average_window)/moving_average_window, mode='valid')
    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(fuel_usage, label='Fuel Usage', alpha=0.5)
    plt.plot(np.arange(moving_average_window - 1, len(fuel_usage)), moving_avg, label='Moving Average', color='red')
    plt.title('Fuel Usage and Moving Average')
    plt.xlabel('Episode')
    plt.ylabel('Fuel Usage')
    plt.legend()
    plt.grid()
    plt.show()

In [66]:
env = gym.make("LunarLander-v3")
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

In [67]:
print(f"State Dimension: {state_dim}, Action Dimension: {action_dim}")

State Dimension: 8, Action Dimension: 4


In [68]:
fuel_cost = {0: 0.0, 1: 0.3, 2: 0.03, 3: 0.03}

In [69]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


## 2.2. Random action

- Trước hết, ta sẽ khởi tạo một lớp có tên `RandomTrain` để kiểm tra các hàm xử lý chung và đảm bảo rằng các chức năng cơ bản của môi trường hoạt động đúng như mong đợi. Việc kiểm tra này giúp phát hiện sớm các lỗi tiềm ẩn trước khi áp dụng các thuật toán huấn luyện phức tạp hơn. Đồng thời, quá trình này cũng cho phép ghi lại và hiển thị một video mẫu minh họa cách tác nhân hoạt động khi hành động được chọn một cách ngẫu nhiên.

In [70]:
class RandomTrain:
    def __init__(self, env, episodes):
        self.env = env
        self.episodes = episodes
        self.rewards = []
        self.losses = []
        self.fuel_usage = []
        self.model = None

    def train(self):
        for episode in tqdm(range(self.episodes)):
            state, _ = self.env.reset()
            total_reward = 0
            total_fuel = 0
            done = False

            while not done:
                action = self.get_action(state)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                total_reward += reward
                total_fuel += fuel_cost[action]
                state = next_state
                done = terminated or truncated

            self.rewards.append(total_reward)
            self.fuel_usage.append(total_fuel)
            #print(f"Episode {episode + 1}/{self.episodes}, Reward: {total_reward}, Fuel Used: {total_fuel}")
            self.env.close()
        return self.rewards, self.losses, self.fuel_usage
    
    def plot_results(self):
        plot_rewards(self.rewards)
        plot_fuel_usage(self.fuel_usage)
        
    def get_action(self, state):
        return np.random.randint(0, action_dim-1)
    
    def display_sample_video(self, sample_episode=1):
        video_render = gym.make("LunarLander-v3", render_mode="rgb_array")
        video_render = RecordVideo(video_render, "videos", episode_trigger=lambda x: True)
        for _ in range(sample_episode):
            state, _ = video_render.reset()
            done = False
            while not done:
                action = np.random.randint(1, 3)
                state, reward, terminated, truncated, _ = video_render.step(action)
                done = terminated or truncated
            video_render.close()
        video_files = glob.glob("videos/*.mp4")
        return Video(video_files[-1])

### Kết quả

In [71]:
# random_train = RandomTrain(env, episodes=500)
# rewards, losses, fuel_usage = random_train.train()
# random_train.plot_results()
# random_train.display_sample_video(sample_episode=1)

# 3. Q-Learning sử dụng Q-Table
---

### a. Giới thiệu

- Q-Learning là một thuật toán học tăng cường không mô hình sử dụng bảng Q-Table để lưu trữ và cập nhật giá trị kỳ vọng của mỗi hành động tại từng trạng thái.
- Q-Table là một bảng trong đó mỗi phần tử `Q(s, a)` biểu thị giá trị kỳ vọng của việc thực hiện hành động `a` tại trạng thái `s` và sau đó tuân theo chính sách tối ưu.
- Vì Q-learning chỉ áp dụng được cho không gian trạng thái `rời rạc`, nên cần lượng tử hóa (discretize) không gian trạng thái của LunarLander `(liên tục)` thành các bins.

### b. Công thức toán học

- Q-Table được cập nhật theo công thức Bellman:
$$
Q(s, a) \leftarrow Q(s, a) + \alpha \left[ r + \gamma \cdot \max_{a'} Q(s', a') - Q(s, a) \right]
$$

Trong đó:

- $s$: trạng thái hiện tại (*current state*)
- $a$: hành động đã chọn (*action taken*)
- $r$: phần thưởng nhận được (*reward received*)
- $s'$: trạng thái tiếp theo (*next state*)
- $\alpha \in (0, 1)$: hệ số học (*learning rate*)
- $\gamma \in (0, 1)$: hệ số chiết khấu (*discount factor*)
- $\max_{a'} Q(s', a')$: giá trị Q tối đa tại trạng thái tiếp theo

### c. Triển khai code

Các hàm chính xử lí thuật toán:

`__init__`

Hàm khởi tạo các tham số cần thiết để huấn luyện:
- Các tham số của thuật toán.
- **bins_size**: Số lượng đoạn chia mỗi chiều trong không gian trạng thái (discretization).
- **state_bins**: Mỗi chiều của trạng thái được chia nhỏ thành các đoạn (bin).
- **q_table**: Q-table 9 chiều chứa giá trị Q cho mỗi tổ hợp trạng thái và hành động. Kích thước:
$$(\text{bins\_size})^6*2*2*\text{số hành động}$$

`get_state_bin(state)`

- Lượng tử hóa (discretize) trạng thái liên tục thành các chỉ số bin rời rạc.
- Dùng np.digitize để xác định chỉ số bin tương ứng.
- Đảm bảo chỉ số nằm trong phạm vi hợp lệ.

`get_action(state)`

Triển khai chính sách ε-greedy:
- Với xác suất ε, chọn hành động ngẫu nhiên (khám phá).
- Ngược lại, chọn hành động có giá trị Q lớn nhất từ Q-table.

`train()`

Hàm huấn luyện tác tử trong nhiều vòng lặp (episode):

1. Mỗi bước:
   - Chọn hành động bằng `get_action(state)`
   - Tương tác với môi trường, nhận phần thưởng $r$
   - Xác định trạng thái kế tiếp $s'$
   - Tính toán sai số TD:

     $$
     \delta = \left[ r + \gamma \cdot \max_{a'} Q(s', a') \right] - Q(s, a)
     $$

   - Cập nhật giá trị Q:

     $$
     Q(s, a) \leftarrow Q(s, a) + \alpha \cdot \delta
     $$

2. Sau mỗi vòng lặp, giảm $\varepsilon$ theo công thức:

   $$
   \varepsilon \leftarrow \max(\varepsilon_{\text{min}}, \varepsilon \cdot \varepsilon_{\text{decay}})
   $$

3. Ghi nhận tổng phần thưởng và mức tiêu hao nhiên liệu.



In [72]:
class QTable:
    def __init__(self, env, episodes, alpha, gamma, epsilon, epsilon_decay, min_epsilon, bins_size):
        self.env = env
        self.episodes = episodes
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.bins_size = bins_size
        self.rewards = []
        self.fuel_usage = []

        self.state_bins = [
            np.linspace(-1.5, 1.5, bins_size),
            np.linspace(-0.5, 1.5, bins_size),
            np.linspace(-2, 2, bins_size),
            np.linspace(-2, 2, bins_size),
            np.linspace(-3.14, 3.14, bins_size),
            np.linspace(-5, 5, bins_size),
            np.array([0, 1]),
            np.array([0, 1]),
        ]

        self.q_table = np.zeros((bins_size, bins_size, bins_size, bins_size,
                                 bins_size, bins_size, 2, 2, action_dim))

    def get_state_bin(self, state):
        state_bins = []
        for i in range(len(state)):
            bin_index = np.digitize(state[i], self.state_bins[i]) - 1
            # Clamp index within valid range
            bin_index = min(max(bin_index, 0), len(self.state_bins[i]) - 1)
            state_bins.append(bin_index)
        return tuple(state_bins)

    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(0, action_dim-1)
        else:
            return np.argmax(self.q_table[self.get_state_bin(state)])

    def train(self):
        for episode in tqdm(range(self.episodes)):
            state, _ = self.env.reset()
            total_reward = 0
            total_fuel = 0
            done = False

            while not done:
                action = self.get_action(state)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                total_reward += reward
                total_fuel += fuel_cost[action]

                # Update Q-table
                state_bin = self.get_state_bin(state)
                next_state_bin = self.get_state_bin(next_state)

                best_next_action = np.argmax(self.q_table[next_state_bin])
                td_target = reward + self.gamma * self.q_table[next_state_bin][best_next_action]
                td_error = td_target - self.q_table[state_bin][action]

                self.q_table[state_bin][action] += self.alpha * td_error
                state = next_state
                done = terminated or truncated

            self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)
            self.rewards.append(total_reward)
            self.fuel_usage.append(total_fuel)

        return self.rewards, [], self.fuel_usage
    
    def plot_results(self):
        plot_rewards(self.rewards)
        plot_fuel_usage(self.fuel_usage)
    
    def display_sample_video(self, sample_episode=5):
        video_render = gym.make("LunarLander-v3", render_mode="rgb_array")
        video_render = RecordVideo(video_render, "videos", episode_trigger=lambda x: True)
        for _ in range(sample_episode):
            state, _ = video_render.reset()
            done = False
            while not done:
                action = self.get_action(state)
                state, reward, terminated, truncated, _ = video_render.step(action)
                done = terminated or truncated
            video_render.close()
        video_files = glob.glob("videos/*.mp4")
        return Video(video_files[-1])
    
    def test_model(self, episodes=500):
        total_success = 0
        testing = gym.make("LunarLander-v3")
        
        for episode in tqdm(range(episodes)):
            state, _ = testing.reset()
            done = False
            total_reward = 0
            self.epsilon = 0
            
            while not done:
                action = self.get_action(state)
                state, reward, terminated, truncated, _ = testing.step(action)
                total_reward += reward
                done = terminated or truncated
            if total_reward >= 200:
                total_success += 1
        print(f"Success: {total_success}/{episodes} | Success Rate: {total_success / episodes * 100:.2f}%")
        testing.close()

### d. Kết quả chính

In [73]:
# train = QTable(env, episodes=5000, alpha=0.1, gamma=0.99, epsilon=1.0, epsilon_decay=0.996, min_epsilon=0.01, bins_size=8)
# rewards, losses, fuel_usage = train.train()

100%|██████████| 5000/5000 [02:31<00:00, 33.07it/s]

In [74]:
# train.plot_results()

![](./images/output(9).png)
![](./images/output(10).png)

In [75]:
# train.test_model(episodes=500)

100%|██████████| 500/500 [00:07<00:00, 62.67it/s]
Success: 19/500 | Success Rate: 3.80%

In [76]:
# train.display_sample_video(sample_episode=5)

### e. Kết luận

- Tỉ lệ thành công chỉ đạt **3.8%** sau 5000 episode và 500 tập kiểm tra cho thấy hiệu quả huấn luyện `Q-table` trên môi trường LunarLander là rất thấp.

- Nguyên nhân có thể: 
1. Không gian trạng thái liên tục và có nhiều chiều

    - Môi trường `LunarLander` có **8 chiều trạng thái liên tục**, khiến việc lượng tử hóa (discretization) trở nên khó khăn. Khi chia mỗi chiều thành `bins_size` đoạn, tổng số trạng thái rời rạc có thể lên tới:

    $$
    \text{bins\_size}^6 \times 2 \times 2
    $$

    - Ví dụ, nếu `bins_size = 10` thì số trạng thái là $10^6 \times 2 \times 2 = 4.000.000$ trạng thái, rất lớn cho bảng Q.

2. Q-table tiêu tốn nhiều bộ nhớ và thời gian học  
    - Khi để bins_size quá lớn sẽ gây ra hiện tượng tràn RAM
    - Không thể khám phá đầy đủ tất cả các trạng thái trong khoảng thời gian huấn luyện hạn chế, dẫn đến việc **overfitting vào số ít trạng thái đã thấy**

3. Không có khả năng tổng quát hóa
   - Q-table chỉ lưu giá trị từng trạng thái cụ thể, không thể chia sẻ thông tin giữa các trạng thái gần nhau. Điều này khiến việc học trong không gian trạng thái liên tục kém hiệu quả.


**$\implies$ Giải quyết: Thay đổi thuật toán, sử dụng `Deep Q-Network` để xấp xỉ hàm Q với mạng neuron, khắc phục nhược điểm tiêu tốn quá nhiều bộ nhớ và không có khả năng tổng quát của `Q-Table`**

# 4. Deep Q-Network
---

### a. Giới thiệu

DQN là thuật toán tăng cường học sâu, nhằm giải quyết bài toán ra quyết định trong môi trường rời rạc, kết hợp:
- Mạng Neuron học sâu
- Q-Learning cổ điển

Khác với Q-learning truyền thống chỉ sử dụng Q-Table để lưu trữ giá trị hành động, DQN sử dụng một mạng nơ-ron để xấp xỉ hàm Q, giúp mở rộng khả năng áp dụng sang các môi trường có không gian trạng thái lớn hoặc liên tục.

Thuật toán DQN hoạt động bằng cách cho tác nhân tương tác với môi trường, thu thập kinh nghiệm dưới dạng các bộ dữ liệu (trạng thái, hành động, phần thưởng, trạng thái kế tiếp), và sử dụng các dữ liệu này để huấn luyện mạng nơ-ron sao cho đầu ra của nó gần đúng với giá trị Q tối ưu. Mục tiêu của DQN là tìm một hàm Q* sao cho tại mỗi trạng thái, hành động được chọn sẽ tối đa hóa tổng phần thưởng trong dài hạn.

### b. Công thức toán học

Tại mỗi bước huấn luyện, DQN tối thiểu hóa hàm mất mát sau:

$$
\mathcal{L}(\theta) = \mathbb{E}_{(s, a, r, s') \sim \mathcal{D}} \left[ \left( y - Q(s, a; \theta) \right)^2 \right]
$$

Trong đó:

- $ \theta $ là tham số của mạng Q hiện tại.  
- $ \mathcal{D} $ là replay buffer (bộ nhớ kinh nghiệm) lưu trữ các trạng thái của môi trường.
- $ y $ là giá trị mục tiêu, xác định bởi công thức: 

$$
y = r + \gamma \cdot \max_{a'} Q(s', a'; \theta^{-})
$$

- $ Q(s, a; \theta) $: giá trị Q được ước lượng bởi mạng hiện tại.  
- $ \theta^{-} $: tham số của mạng Q mục tiêu (target network), được cập nhật định kỳ từ $ \theta $.  
- $ \gamma \in [0, 1] $: hệ số chiết khấu phần thưởng tương lai.


Việc sử dụng `Replay Buffer` và `Target Q-Net` là hai cải tiến chính giúp DQN ổn định hơn và tránh hiện tượng học lệch, từ đó cho kết quả huấn luyện hiệu quả hơn so với các phương pháp Q-learning truyền thống.




### c. Triển khai code

Lớp `DQNetwork` là một mạng nơ-ron sâu (deep neural network) được sử dụng để ước lượng hàm Q trong thuật toán Deep Q-Learning (DQN).

Mạng này bao gồm:

- Một lớp đầu vào với kích thước bằng số chiều trạng thái (`state_dim`).
- Một lớp ẩn, mỗi lớp gồm 256 nút và sử dụng hàm kích hoạt ReLU.
- Một lớp đầu ra với kích thước bằng số hành động (`action_dim`), trả về giá trị Q tương ứng với từng hành động.

In [77]:
class DQNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(state_dim, 256) ,
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, action_dim)
        )
    def forward(self, x):
        return self.model(x)

Lớp `DQNTrain` chịu trách nhiệm huấn luyện agent sử dụng thuật toán Deep Q-Network (DQN) trên môi trường như LunarLander-v3. Nó bao gồm các thành phần chính như sau:

**Khởi tạo mô hình**
- Khởi tạo mạng Q chính (`q_net`) và mạng Q mục tiêu (`target_net`).
- Khởi tạo replay buffer để lưu trữ kinh nghiệm.
- Chọn thuật toán tối ưu hóa (````adam````, RMSprop hoặc SGD).
- Thiết lập các siêu tham số như epsilon, gamma, batch size, số tập huấn luyện,...

**Chọn hành động (`select_action`)**
- Dựa trên chính sách ε-greedy: chọn ngẫu nhiên hoặc hành động tốt nhất theo Q hiện tại. Chính sách này kết hợp giữa hành động ngẫu nhiên và hành động tối ưu:
    - Với xác suất ε (epsilon), chọn hành động ngẫu nhiên.

    - Với xác suất 1 - ε, chọn hành động tốt nhất (hành động có giá trị Q cao nhất).

- Mục đích của ε-greedy là khuyến khích việc khám phá không gian trạng thái trong giai đoạn đầu và sau đó chuyển sang khai thác những hành động tối ưu đã học được khi epsilon giảm dần.

**Huấn luyện từng bước (`train_step`)**
- Lấy minibatch từ replay buffer.
- Tính toán giá trị Q hiện tại và Q mục tiêu.
- Tối ưu hóa bằng hàm mất mát MSE để cập nhật mạng Q.

**Vòng lặp huấn luyện (`train`)**
- Lặp qua các episode, cập nhật mạng mục tiêu định kỳ.
- Theo dõi reward, loss và mức tiêu hao nhiên liệu qua từng tập.

Các chú thích chi tiết ở trong code sau:


In [78]:
class DQNTrain:
    def __init__(self, env, optimizer, epsilon, min_epsilon, decay, gamma, batch_size, episodes, target_update_freq, memory_size, learning_rate):
        '''
        - Khởi tạo các tham số cho DQN
        - Khởi tạo optimizer (`adam`, RMSprop, SGD)
        - Khởi tạo mạng Q và mạng Q mục tiêu
        - Khởi tạo bộ nhớ (Replay Buffer) để lưu trữ các trạng thái của môi trường
        - Khởi tạo các biến để theo dõi phần thưởng, tổn thất và mức tiêu thụ nhiên liệu
        '''
        self.env = env # Môi trường
        self.episodes = episodes # Số lượng tập huấn luyện
        self.epsilon = epsilon # Giá trị epsilon ban đầu
        self.min_epsilon = min_epsilon # Giá trị epsilon tối thiểu
        self.decay = decay # Hệ số giảm epsilon
        self.gamma = gamma # Hệ số giảm giá cho phần thưởng
        self.batch_size = batch_size # Kích thước batch cho việc huấn luyện
        self.target_update_freq = target_update_freq # Tần suất cập nhật mạng Q mục tiêu
        self.memory = deque(maxlen=memory_size) # Bộ nhớ để lưu trữ các trạng thái (replay buffer)
        self.fuel_usage = [] # Danh sách để lưu trữ mức tiêu thụ nhiên liệu
        self.rewards = [] # Danh sách để lưu trữ phần thưởng
        self.losses = [] # Danh sách để lưu trữ loss
        
        self.q_net = DQNetwork(state_dim, action_dim).to(device)
        self.target_net = DQNetwork(state_dim, action_dim).to(device)
        self.target_net.load_state_dict(self.q_net.state_dict())
        self.target_net.eval()
        
        if optimizer == 'adam':
            self.optimizer = torch.optim.Adam(self.q_net.parameters(), lr=learning_rate)
        elif optimizer == 'rmsprop':
            self.optimizer = torch.optim.RMSprop(self.q_net.parameters(), lr=learning_rate)
        elif optimizer == 'sgd':
            self.optimizer = torch.optim.SGD(self.q_net.parameters(), lr=learning_rate)
        
        
    def select_action(self, state):
        '''
        - Chọn hành động dựa trên epsilon-greedy
        + Nếu giá trị ngẫu nhiên nhỏ hơn epsilon, chọn hành động ngẫu nhiên
        + Ngược lại, chọn hành động có giá trị Q lớn nhất từ mạng Q
        - Trả về chỉ số của hành động được chọn 
        '''
        if random.random() < self.epsilon:
            return random.randint(0, action_dim - 1)
        else:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
                return self.q_net(state_tensor).argmax().item()
            

    def train_step(self):
        ''''
        - Huấn luyện mạng Q bằng cách lấy mẫu ngẫu nhiên từ bộ nhớ, nếu bộ nhớ lớn hơn kích thước batch
        - Chuyển đổi các trạng thái, hành động, phần thưởng, trạng thái tiếp theo và done thành tensor
        - Tính toán giá trị Q hiện tại và giá trị Q mục tiêu
        - Tính toán loss bằng cách sử dụng hàm mất mát MSE
        - Cập nhật trọng số của mạng Q bằng cách sử dụng thuật toán tối ưu hóa
        - Trả về giá trị loss
        '''
        
        if len(self.memory) < self.batch_size:
            return 0
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = np.array(states)
        actions = np.array(actions)
        rewards = np.array(rewards)
        next_states = np.array(next_states)
        dones = np.array(dones)

        states = torch.FloatTensor(np.array(states)).to(device)
        actions = torch.LongTensor(actions).unsqueeze(1).to(device)
        rewards = torch.FloatTensor(rewards).unsqueeze(1).to(device)
        next_states = torch.FloatTensor(np.array(next_states)).to(device)
        dones = torch.FloatTensor(dones).unsqueeze(1).to(device)

        q_values = self.q_net(states).gather(1, actions)

        with torch.no_grad():
            max_next_q = self.target_net(next_states).max(1)[0].unsqueeze(1)
            targets = rewards + self.gamma * max_next_q * (1 - dones)

        loss = nn.MSELoss()(q_values, targets)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()

    def train(self):
        '''
        - Huấn luyện DQN bằng cách lặp qua số lượng tập huấn luyện
        - Trong mỗi tập, khởi tạo trạng thái và phần thưởng
        - Lặp qua các bước trong mỗi tập cho đến khi hoàn thành
        - Chọn hành động bằng cách sử dụng hàm select_action
        - Thực hiện hành động và nhận trạng thái tiếp theo, phần thưởng và trạng thái hoàn thành
        - Lưu trữ trạng thái vào bộ nhớ
        - Tính toán loss bằng cách sử dụng hàm train_step
        - Cập nhật mạng Q mục tiêu theo tần suất đã chỉ định
        - Lưu trữ phần thưởng, loss và mức tiêu thụ nhiên liệu vào danh sách
        - Trả về danh sách phần thưởng, loss và mức tiêu thụ nhiên liệu
        '''
        
        pbar = tqdm(range(self.episodes), desc="Training DQN")
        
        for ep in pbar:
            state, _ = self.env.reset()
            total_reward = 0
            total_loss = 0
            total_fuel = 0
            done = False

            while not done:
                action = self.select_action(state)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                self.memory.append((state, action, reward, next_state, float(done)))
                
                loss = self.train_step()
                total_loss += loss
                total_reward += reward
                total_fuel += fuel_cost[action]
                
                state = next_state

            if ep % self.target_update_freq == 0:
                self.target_net.load_state_dict(self.q_net.state_dict())
                
            if ep % 100 == 0:
                average_reward = np.mean(self.rewards[-100:])
                print(f"Episode {ep}, Average Reward: {average_reward:.2f}")
                
            self.epsilon = max(self.min_epsilon, self.epsilon * self.decay)
            self.rewards.append(total_reward)
            self.losses.append(total_loss)
            self.fuel_usage.append(total_fuel)

            pbar.set_postfix({
                "Reward": total_reward,
                "Epsilon": self.epsilon,
                "Loss": total_loss,
                "Fuel Usage": total_fuel
            })

        return self.rewards, self.losses, self.fuel_usage


    def save_model(self, path):
        '''
        - Lưu trọng số của mạng Q vào tệp
        '''
        torch.save(self.target_net.state_dict(), path)


    def load_model(self, path):
        '''
        - Tải trọng số của mạng Q từ tệp
        - Đặt mạng Q mục tiêu ở chế độ đánh giá để không cập nhật trọng số trong quá trình kiểm tra
        '''
        self.target_net.load_state_dict(torch.load(path))
        self.target_net.eval()
        
        
    def get_target_action(self, state):
        ''' 
        - Chọn hành động từ mạng Q mục tiêu để kiểm tra
        '''
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            return self.target_net(state_tensor).argmax().item()


    def plot_results(self):
        '''
        - Vẽ biểu đồ phần thưởng, tổn thất và mức tiêu thụ nhiên liệu
        - Sử dụng các hàm plot_rewards, plot_losses và plot_fuel_usage đã định nghĩa ở trên
        '''
        plot_rewards(self.rewards)
        plot_losses(self.losses)
        plot_fuel_usage(self.fuel_usage)
        
    
    def test_model(self, episodes=500):
        '''
        - Kiểm tra mô hình đã được huấn luyện bằng cách chạy một số tập
        - Trong mỗi tập, khởi tạo trạng thái và phần thưởng
        - Lặp qua các bước trong mỗi tập cho đến khi hoàn thành
        - Chọn hành động bằng cách sử dụng hàm get_target_action
        - Thực hiện hành động và nhận trạng thái tiếp theo, phần thưởng và trạng thái hoàn thành
        - Tính toán tổng phần thưởng
        - Nếu tổng phần thưởng lớn hơn hoặc bằng 200, tăng biến thành công
        - Trả về số lượng thành công và tỷ lệ thành công
        '''
        total_success = 0
        testing = gym.make("LunarLander-v3")
        
        for episode in tqdm(range(episodes)):
            state, _ = testing.reset()
            done = False
            total_reward = 0
            
            while not done:
                action = self.get_target_action(state)
                state, reward, terminated, truncated, _ = testing.step(action)
                total_reward += reward
                done = terminated or truncated
            if total_reward >= 200:
                total_success += 1
        print(f"Success: {total_success}/{episodes} | Success Rate: {total_success / episodes * 100:.2f}%")
        testing.close()
        
        
    def display_sample_video(self, sample_video=1):
        '''
        - Hiển thị video mẫu của mô hình đã được huấn luyện
        - Sử dụng RecordVideo để ghi lại video trong môi trường
        - Trong mỗi tập, khởi tạo trạng thái và phần thưởng
        - Lặp qua các bước trong mỗi tập cho đến khi hoàn thành
        - Chọn hành động bằng cách sử dụng hàm get_target_action
        - Thực hiện hành động và nhận trạng thái tiếp theo, phần thưởng và trạng thái hoàn thành
        - Đóng video sau khi hoàn thành
        - Trả về video cuối cùng được ghi lại
        '''
        video_render = gym.make("LunarLander-v3", render_mode="rgb_array")
        video_render = RecordVideo(video_render, "videos", episode_trigger=lambda x: True)
        
        for _ in tqdm(range(sample_video)):
            state, _ = video_render.reset()
            done = False
            while not done:
                action = self.get_target_action(state)
                state, reward, terminated, truncated, _ = video_render.step(action)
                done = terminated or truncated
            video_render.close()
        video_files = glob.glob("videos/*.mp4")
        return Video(video_files[-1])

### d. Kết quả chính

#### Kết quả với tối ưu RMSPROP

In [79]:
# trainer = DQNTrain(
#     env=env,
#     optimizer='rmsprop',
#     epsilon=1.0,
#     min_epsilon=0.01,
#     decay=0.995,
#     gamma=0.99,
#     batch_size=128,
#     episodes=2000,
#     target_update_freq=10,
#     memory_size=100000,
#     learning_rate=1e-4
# )

In [80]:
# rewards, losses, fuel_usage = trainer.train()

Training DQN:   0%|          | 0/2000 [00:00<?, ?it/s, Reward=-7.56, Epsilon=0.995, Loss=0, Fuel Usage=8.16]
Episode 0, Average Reward: nan
Training DQN:   5%|▌         | 101/2000 [00:28<11:40,  2.71it/s, Reward=-92.7, Epsilon=0.603, Loss=3.14e+3, Fuel Usage=8.31]
Episode 100, Average Reward: -140.42
Training DQN:  10%|█         | 201/2000 [02:01<1:11:47,  2.39s/it, Reward=-49.2, Epsilon=0.365, Loss=1.19e+4, Fuel Usage=82.6]
Episode 200, Average Reward: -58.85
Training DQN:  15%|█▌        | 301/2000 [06:08<1:15:53,  2.68s/it, Reward=-14.8, Epsilon=0.221, Loss=3.99e+3, Fuel Usage=62.1]   
Episode 300, Average Reward: -22.66
Training DQN:  20%|██        | 401/2000 [09:57<1:00:30,  2.27s/it, Reward=146, Epsilon=0.134, Loss=4.82e+3, Fuel Usage=49.7]   
Episode 400, Average Reward: 33.97
Training DQN:  25%|██▌       | 501/2000 [13:39<49:22,  1.98s/it, Reward=12.3, Epsilon=0.0812, Loss=2.98e+3, Fuel Usage=21.6]   
Episode 500, Average Reward: 32.39
Training DQN:  30%|███       | 601/2000 [16:53<52:56,  2.27s/it, Reward=-32.7, Epsilon=0.0492, Loss=8.74e+3, Fuel Usage=62.2]
Episode 600, Average Reward: 104.68
Training DQN:  35%|███▌      | 701/2000 [19:52<30:53,  1.43s/it, Reward=242, Epsilon=0.0298, Loss=4.9e+3, Fuel Usage=29.4]   
Episode 700, Average Reward: 117.25
Training DQN:  40%|████      | 801/2000 [22:26<19:48,  1.01it/s, Reward=238, Epsilon=0.018, Loss=2.47e+3, Fuel Usage=14.7]   
Episode 800, Average Reward: 181.27
Training DQN:  45%|████▌     | 901/2000 [24:09<14:24,  1.27it/s, Reward=254, Epsilon=0.0109, Loss=2.18e+3, Fuel Usage=12.2]
Episode 900, Average Reward: 240.24
Training DQN:  50%|█████     | 1001/2000 [25:51<16:08,  1.03it/s, Reward=257, Epsilon=0.01, Loss=1.98e+3, Fuel Usage=10.9] 
Episode 1000, Average Reward: 237.16
Training DQN:  55%|█████▌    | 1101/2000 [27:23<12:54,  1.16it/s, Reward=258, Epsilon=0.01, Loss=1.14e+3, Fuel Usage=8.82]  
Episode 1100, Average Reward: 244.40
Training DQN:  60%|██████    | 1201/2000 [28:39<14:11,  1.07s/it, Reward=210, Epsilon=0.01, Loss=4.43e+3, Fuel Usage=88.9]
Episode 1200, Average Reward: 260.15
Training DQN:  65%|██████▌   | 1301/2000 [29:52<09:03,  1.29it/s, Reward=253, Epsilon=0.01, Loss=1.65e+3, Fuel Usage=12]  
Episode 1300, Average Reward: 259.16
Training DQN:  70%|███████   | 1401/2000 [31:02<06:07,  1.63it/s, Reward=257, Epsilon=0.01, Loss=1.65e+3, Fuel Usage=15.1]
Episode 1400, Average Reward: 247.33
Training DQN:  75%|███████▌  | 1501/2000 [32:15<06:23,  1.30it/s, Reward=278, Epsilon=0.01, Loss=1.6e+3, Fuel Usage=15.5]   
Episode 1500, Average Reward: 245.86
Training DQN:  80%|████████  | 1601/2000 [33:31<04:20,  1.53it/s, Reward=289, Epsilon=0.01, Loss=1.31e+3, Fuel Usage=15.6] 
Episode 1600, Average Reward: 256.02
Training DQN:  85%|████████▌ | 1701/2000 [34:53<04:55,  1.01it/s, Reward=287, Epsilon=0.01, Loss=1.24e+3, Fuel Usage=17.5]
Episode 1700, Average Reward: 252.03
Training DQN:  90%|█████████ | 1801/2000 [36:07<01:56,  1.70it/s, Reward=247, Epsilon=0.01, Loss=732, Fuel Usage=7.65]    
Episode 1800, Average Reward: 259.45
Training DQN:  95%|█████████▌| 1901/2000 [37:18<00:57,  1.71it/s, Reward=291, Epsilon=0.01, Loss=994, Fuel Usage=15.5]    
Episode 1900, Average Reward: 260.10
Training DQN: 100%|██████████| 2000/2000 [38:25<00:00,  1.15s/it, Reward=237, Epsilon=0.01, Loss=1.84e+3, Fuel Usage=13.5] 

In [81]:
# trainer.plot_results()

![i1](images/output.png)
![i2](images/output(1).png)
![i3](images/output(2).png)

In [82]:
#trainer.save_model("dqn_lunarlander-2k-rmsprop.pth")

In [83]:
#trainer.load_model("dqn_lunarlander-2k-rmsprop.pth")

In [84]:
#trainer.test_model(episodes=500)

Success: 414/500 | Success Rate: 82.80%

In [85]:
#trainer.display_sample_video(sample_video=5)

In [86]:
Video("videos-rmsprop/rl-video-episode-4.mp4")

#### Kết quả với tối ưu Adam

In [87]:
# trainer = DQNTrain(
#     env=env,
#     optimizer='adam',
#     epsilon=1.0,
#     min_epsilon=0.01,
#     decay=0.995,
#     gamma=0.99,
#     batch_size=128,
#     episodes=2000,
#     target_update_freq=10,
#     memory_size=100000,
#     learning_rate=1e-4
# )

In [88]:
#rewards, losses, fuel_usage = trainer.train()

Training DQN:   0%|          | 2/2000 [00:00<03:17, 10.13it/s, Reward=-161, Epsilon=0.99, Loss=3.35e+3, Fuel Usage=9.63]
Episode 0, Average Reward: nan
Training DQN:   5%|▌         | 101/2000 [00:29<14:36,  2.17it/s, Reward=-142, Epsilon=0.603, Loss=2.65e+3, Fuel Usage=11.7] 
Episode 100, Average Reward: -119.58
Training DQN:  10%|█         | 201/2000 [02:08<1:06:03,  2.20s/it, Reward=-46.4, Epsilon=0.365, Loss=1e+4, Fuel Usage=68.4]   
Episode 200, Average Reward: -68.83
Training DQN:  15%|█▌        | 301/2000 [06:27<1:16:05,  2.69s/it, Reward=-14.3, Epsilon=0.221, Loss=3.74e+3, Fuel Usage=63.6]
Episode 300, Average Reward: -16.85
Training DQN:  20%|██        | 401/2000 [10:18<1:07:34,  2.54s/it, Reward=70.8, Epsilon=0.134, Loss=5.68e+3, Fuel Usage=48.4] 
Episode 400, Average Reward: 47.01
Training DQN:  25%|██▌       | 501/2000 [13:52<33:58,  1.36s/it, Reward=249, Epsilon=0.0812, Loss=2.61e+3, Fuel Usage=9.39]    
Episode 500, Average Reward: 74.07
Training DQN:  30%|███       | 601/2000 [16:42<31:56,  1.37s/it, Reward=234, Epsilon=0.0492, Loss=3.54e+3, Fuel Usage=7.86]   
Episode 600, Average Reward: 152.28
Training DQN:  35%|███▌      | 701/2000 [19:09<19:15,  1.12it/s, Reward=279, Epsilon=0.0298, Loss=2.73e+3, Fuel Usage=11.9]  
Episode 700, Average Reward: 164.76
Training DQN:  40%|████      | 801/2000 [21:01<19:31,  1.02it/s, Reward=279, Epsilon=0.018, Loss=3.23e+3, Fuel Usage=9.9]    
Episode 800, Average Reward: 221.37
Training DQN:  45%|████▌     | 901/2000 [22:35<15:02,  1.22it/s, Reward=296, Epsilon=0.0109, Loss=2.52e+3, Fuel Usage=11.7]  
Episode 900, Average Reward: 233.87
Training DQN:  50%|█████     | 1001/2000 [23:55<13:15,  1.26it/s, Reward=283, Epsilon=0.01, Loss=2.01e+3, Fuel Usage=12.6] 
Episode 1000, Average Reward: 260.68
Training DQN:  55%|█████▌    | 1101/2000 [25:12<11:50,  1.27it/s, Reward=234, Epsilon=0.01, Loss=1.44e+3, Fuel Usage=20.4]  
Episode 1100, Average Reward: 259.33
Training DQN:  60%|██████    | 1201/2000 [26:31<08:43,  1.53it/s, Reward=266, Epsilon=0.01, Loss=1.78e+3, Fuel Usage=9.48]  
Episode 1200, Average Reward: 255.35
Training DQN:  65%|██████▌   | 1301/2000 [27:37<06:49,  1.71it/s, Reward=287, Epsilon=0.01, Loss=1.04e+3, Fuel Usage=10.5]
Episode 1300, Average Reward: 266.93
Training DQN:  70%|███████   | 1401/2000 [28:38<05:51,  1.71it/s, Reward=245, Epsilon=0.01, Loss=1.12e+3, Fuel Usage=9.36]
Episode 1400, Average Reward: 273.05
Training DQN:  75%|███████▌  | 1501/2000 [29:40<04:19,  1.92it/s, Reward=276, Epsilon=0.01, Loss=813, Fuel Usage=7.26]    
Episode 1500, Average Reward: 278.39
Training DQN:  80%|████████  | 1601/2000 [30:41<03:21,  1.98it/s, Reward=270, Epsilon=0.01, Loss=982, Fuel Usage=4.11]    
Episode 1600, Average Reward: 277.69
Training DQN:  85%|████████▌ | 1701/2000 [31:43<02:36,  1.91it/s, Reward=302, Epsilon=0.01, Loss=1.78e+3, Fuel Usage=10.3]
Episode 1700, Average Reward: 267.48
Training DQN:  90%|█████████ | 1801/2000 [32:41<01:48,  1.84it/s, Reward=308, Epsilon=0.01, Loss=1.09e+3, Fuel Usage=9.18]
Episode 1800, Average Reward: 255.86
Training DQN:  95%|█████████▌| 1901/2000 [33:36<00:48,  2.05it/s, Reward=252, Epsilon=0.01, Loss=854, Fuel Usage=6.69]     
Episode 1900, Average Reward: 248.64
Training DQN: 100%|██████████| 2000/2000 [34:30<00:00,  1.04s/it, Reward=276, Epsilon=0.01, Loss=1.87e+3, Fuel Usage=14.4]  

In [89]:
#trainer.plot_results()

![](images/output(3).png)
![](images/output(4).png)
![](images/output(5).png)

In [90]:
#trainer.save_model("dqn_lunarlander-2k-adam.pth")

In [91]:
#trainer.load_model("dqn_lunarlander-2k-adam.pth")

In [92]:
#trainer.test_model(episodes=500)

Success: 484/500 | Success Rate: 96.80%

In [93]:
#trainer.display_sample_video(sample_video=5)

In [94]:
Video("videos-adam/rl-video-episode-4.mp4")

#### **So sánh**

Với cùng thông số, `adam` cho kết quả tốt hơn so với `rmsprop`
- Average Reward của `adam` đạt trên 200 ep700-800 còn `rmsprop` là ep800-900
- Độ chính xác khi hạ cánh của `adam` rất cao khi đạt tỉ lệ 96.80% trong khi `rmsprop` là 82.80%
- Thời gian chạy cũng nhanh hơn khi `adam` chỉ mất 34p còn `rmsprop` mất 38p

$\implies$ Lý do:
- `adam` tích hợp cả momentum và `rmsprop` nên hiệu quả học thường cao hơn đối với môi trường phức tạp
- Môi trường ổn định (không gió, không nhiễu) nên cần một thuật toán có thể khiến loss mô hình hội tụ nhanh như `adam`

#### Adam với learning rate khác

In [95]:
# trainer = DQNTrain(
#     env=env,
#     optimizer='adam',
#     epsilon=1.0,
#     min_epsilon=0.01,
#     decay=0.995,
#     gamma=0.99,
#     batch_size=128,
#     episodes=2000,
#     target_update_freq=10,
#     memory_size=100000,
#     learning_rate=1e-3
# )

In [96]:
# rewards, losses, fuel_usage = trainer.train()

Training DQN:   0%|          | 0/2000 [00:00<?, ?it/s, Reward=-88.5, Epsilon=0.995, Loss=0, Fuel Usage=7.5]
Episode 0, Average Reward: nan
Training DQN:   5%|▌         | 101/2000 [00:28<10:40,  2.97it/s, Reward=-75.1, Epsilon=0.603, Loss=2.69e+3, Fuel Usage=7.08]
Episode 100, Average Reward: -136.95
Training DQN:  10%|█         | 201/2000 [01:43<31:42,  1.06s/it, Reward=-154, Epsilon=0.365, Loss=1.29e+4, Fuel Usage=51.2] 
Episode 200, Average Reward: -80.68
Training DQN:  15%|█▌        | 301/2000 [05:02<44:12,  1.56s/it, Reward=-116, Epsilon=0.221, Loss=6.74e+3, Fuel Usage=26.3]   
Episode 300, Average Reward: -110.11
Training DQN:  20%|██        | 401/2000 [08:44<36:53,  1.38s/it, Reward=-40.3, Epsilon=0.134, Loss=1.28e+3, Fuel Usage=11.8]  
Episode 400, Average Reward: -68.13
Training DQN:  25%|██▌       | 501/2000 [11:03<18:17,  1.37it/s, Reward=248, Epsilon=0.0812, Loss=4.29e+3, Fuel Usage=15.4]   
Episode 500, Average Reward: 19.42
Training DQN:  30%|███       | 602/2000 [11:41<05:16,  4.41it/s, Reward=-215, Epsilon=0.0489, Loss=1.11e+3, Fuel Usage=4.35] 
Episode 600, Average Reward: -150.49
Training DQN:  35%|███▌      | 701/2000 [12:08<05:46,  3.75it/s, Reward=-208, Epsilon=0.0298, Loss=2.45e+3, Fuel Usage=8.43] 
Episode 700, Average Reward: -192.99
Training DQN:  40%|████      | 801/2000 [12:41<08:00,  2.50it/s, Reward=48.7, Epsilon=0.018, Loss=4.36e+3, Fuel Usage=4.56]  
Episode 800, Average Reward: -55.63
Training DQN:  45%|████▌     | 901/2000 [14:05<21:35,  1.18s/it, Reward=186, Epsilon=0.0109, Loss=3.02e+4, Fuel Usage=59.9]  
Episode 900, Average Reward: -1.02
Training DQN:  50%|█████     | 1001/2000 [15:49<19:03,  1.14s/it, Reward=236, Epsilon=0.01, Loss=1.38e+4, Fuel Usage=11.1]   
Episode 1000, Average Reward: 30.97
Training DQN:  55%|█████▌    | 1101/2000 [17:38<15:28,  1.03s/it, Reward=-124, Epsilon=0.01, Loss=6.45e+3, Fuel Usage=6.96] 
Episode 1100, Average Reward: 48.55
Training DQN:  60%|██████    | 1201/2000 [19:21<11:32,  1.15it/s, Reward=272, Epsilon=0.01, Loss=1.08e+4, Fuel Usage=23.3]  
Episode 1200, Average Reward: -50.40
Training DQN:  65%|██████▌   | 1301/2000 [21:04<12:22,  1.06s/it, Reward=271, Epsilon=0.01, Loss=9.94e+3, Fuel Usage=24.7]  
Episode 1300, Average Reward: 14.16
Training DQN:  70%|███████   | 1401/2000 [22:50<10:03,  1.01s/it, Reward=302, Epsilon=0.01, Loss=1.15e+4, Fuel Usage=13.2]  
Episode 1400, Average Reward: 115.29
Training DQN:  75%|███████▌  | 1501/2000 [24:50<08:00,  1.04it/s, Reward=-102, Epsilon=0.01, Loss=3.91e+3, Fuel Usage=16.3] 
Episode 1500, Average Reward: 134.05
Training DQN:  80%|████████  | 1601/2000 [26:40<05:56,  1.12it/s, Reward=263, Epsilon=0.01, Loss=4.96e+3, Fuel Usage=8.25]     
Episode 1600, Average Reward: 93.36
Training DQN:  85%|████████▌ | 1701/2000 [28:14<03:33,  1.40it/s, Reward=-8.37, Epsilon=0.01, Loss=7.22e+3, Fuel Usage=13.9]
Episode 1700, Average Reward: -6.29
Training DQN:  90%|█████████ | 1801/2000 [29:39<02:46,  1.19it/s, Reward=194, Epsilon=0.01, Loss=1.23e+4, Fuel Usage=12.9]     
Episode 1800, Average Reward: -190.80
Training DQN:  95%|█████████▌| 1901/2000 [30:54<01:09,  1.42it/s, Reward=-251, Epsilon=0.01, Loss=6.05e+3, Fuel Usage=1.98]    
Episode 1900, Average Reward: -71.62
Training DQN: 100%|██████████| 2000/2000 [32:28<00:00,  1.03it/s, Reward=264, Epsilon=0.01, Loss=1.76e+4, Fuel Usage=12.2]  

In [97]:
# trainer.plot_results()

![](./images/output(6).png)
![](./images/output(7).png)
![](./images/output(8).png)


In [98]:
# trainer.save_model("dqn_lunarlander-2k-adam-1e-3.pth")

In [99]:
# trainer.load_model("dqn_lunarlander-2k-adam-1e-3.pth")

In [100]:
# trainer.test_model(episodes=500)

Success: 187/500 | Success Rate: 37.40%

In [101]:
# trainer.display_sample_video(sample_video=5)

In [102]:
Video("videos-adam-1e-3/rl-video-episode-4.mp4")

#### **So sánh**

Với cùng thuật toán tối ưu `adam`, thay đổi `learning_rate` từ (1e-4) sang (1e-3).
Kết quả cho thấy: 
- Success rate của 1e-3 đạt 37.2%, hiệu suất giảm rõ rệt so với 96.8% (1e-4).
- Thời gian training (kì vọng giảm rõ rệt) nhưng mất đến 32p, gần tương đương với 34p của (1e-4) mà hiệu suất giảm rõ rệt.
- Điểm thưởng không ổn định như (1e-4).
- Loss cũng cho thấy sự không ổn định của (1e-3) (có thể là do vấn đề liên quan đến exploding-gradient khi loss tăng cao).

**$\implies$ Việc giảm `learning_rate` xuống thấp có thể khiến hiệu suất mô hình giảm trong khi thời gian training mô hình không thay đổi đáng kể**

### e. Kết luận

- Trong môi trường rời rạc tốt (không gió, không nhiễu), thuật toán DQN hoạt động rất hiệu quả có thể đạt hiệu suất trên **99%** nếu training episodes đủ nhiều, có thể sử dụng `lunarlander-5k.pth` để chứng minh hiệu suất.
- Lý do tại sao DQN lại hoạt động hiệu suất cao như vậy:
    - Không gian hành động nhỏ: Với chỉ 4 hành động rời rạc (không làm gì, bắn động cơ chính, bắn động cơ bên trái, bắn động cơ bên phải), DQN có thể dễ dàng học và cập nhật hàm Q cho từng hành động.
    - Tập trạng thái có thể được biểu diễn tốt qua mạng neuron: Trạng thái gồm 8 giá trị thực (tọa độ, vận tốc, góc, trạng thái tiếp xúc), giúp mạng học các đặc trưng quan trọng mà không cần xử lý dữ liệu phức tạp như hình ảnh.
    - Phản hồi phần thưởng rõ ràng, có cấu trúc: Phần thưởng trong LunarLander được thiết kế khuyến khích đáp xuống nhẹ nhàng và phạt cho các hành vi nguy hiểm, giúp DQN dễ học thông qua phản hồi.
    - Không có yếu tố ngẫu nhiên mạnh (như gió, nhiễu loạn): Việc môi trường có tính ổn định tương đối cao giúp mạng học dễ dàng và ổn định hơn.
    - Sử dụng các kĩ thuật `Replay Buffer` và `Target Q-Network` giúp mô hình học ổn định hơn.

In [103]:
# # Chứng minh hiệu suất của mô hình đã được huấn luyện 5k episodes
# test = DQNTrain(
#     env=env,
#     optimizer='adam',
#     epsilon=1.0,
#     min_epsilon=0.01,
#     decay=0.995,
#     gamma=0.99,
#     batch_size=128,
#     episodes=2000,
#     target_update_freq=10,
#     memory_size=100000,
#     learning_rate=1e-4
# )

# test.load_model("lunarlander-5k.pth")
# test.test_model(episodes=1000)

# 5. Double Deep Q-Network
---

Với mô hình LunarLander truyền thống, khi không có sự tác động mạnh của môi trường như gió hoặc nhiễu loạn gió, đặc trưng học nhanh của mô hình (đặc biệt khi sử dụng thuật toán tối ưu `Adam`) giúp `DQN` đạt hiệu suất rất cao. Điều này có thể là do:
- **Không gian trạng thái ổn định và dễ dự đoán**: Khi loại bỏ các yếu tố gây nhiễu như gió, môi trường trở nên nhất quán hơn giữa các lần huấn luyện. Điều này giúp mạng neuron trong DQN dễ dàng học được các quy luật điều khiển phù hợp, mà không cần phải xử lý các tình huống ngoại lệ hay biến động bất thường.

## a. Môi trường gió

Chúng ta sẽ test thử mô hình DQN với môi trường nhiễu động do gió:
- wind_power=15 (tốc độ gió)
- turbulence_power=1.5 (nhiễu loạn gió)

In [104]:
win_env = gym.make("LunarLander-v3",gravity=-10.0, enable_wind=True, wind_power=15.0, turbulence_power=1.5)

## b. DQN Training

In [105]:
win_env_dqn_train = DQNTrain(
    env=win_env,
    optimizer='adam',
    epsilon=1.0,
    min_epsilon=0.01,
    decay=0.995,
    gamma=0.99,
    batch_size=128,
    episodes=2000,
    target_update_freq=10,
    memory_size=100000,
    learning_rate=1e-4
)

In [106]:
# reward, losses, fuel_usage = win_env_dqn_train.train()

Training DQN:   0%|          | 2/2000 [00:00<03:24,  9.76it/s, Reward=-121, Epsilon=0.99, Loss=5.15e+3, Fuel Usage=10.6]
Episode 0, Average Reward: nan
Training DQN:   5%|▌         | 101/2000 [00:25<09:03,  3.50it/s, Reward=-209, Epsilon=0.603, Loss=2.18e+3, Fuel Usage=12.4] 
Episode 100, Average Reward: -238.64
Training DQN:  10%|█         | 201/2000 [01:07<11:44,  2.56it/s, Reward=-58.4, Epsilon=0.365, Loss=1.59e+3, Fuel Usage=8.85] 
Episode 200, Average Reward: -146.54
Training DQN:  15%|█▌        | 301/2000 [02:40<1:05:00,  2.30s/it, Reward=-79.9, Epsilon=0.221, Loss=1.43e+4, Fuel Usage=89.9]
Episode 300, Average Reward: -97.41
Training DQN:  20%|██        | 401/2000 [04:48<23:30,  1.13it/s, Reward=-84, Epsilon=0.134, Loss=2.94e+3, Fuel Usage=14.9]    
Episode 400, Average Reward: -5.58
Training DQN:  25%|██▌       | 501/2000 [06:53<43:47,  1.75s/it, Reward=125, Epsilon=0.0812, Loss=1.43e+4, Fuel Usage=75.4]  
Episode 500, Average Reward: 17.24
Training DQN:  30%|███       | 601/2000 [08:55<21:45,  1.07it/s, Reward=-42.4, Epsilon=0.0492, Loss=7.16e+3, Fuel Usage=18]  
Episode 600, Average Reward: 47.06
Training DQN:  35%|███▌      | 701/2000 [11:24<36:02,  1.66s/it, Reward=222, Epsilon=0.0298, Loss=9.36e+3, Fuel Usage=45.6]  
Episode 700, Average Reward: 107.95
Training DQN:  40%|████      | 801/2000 [14:18<35:43,  1.79s/it, Reward=-123, Epsilon=0.018, Loss=1.36e+4, Fuel Usage=75.9]  
Episode 800, Average Reward: 115.25
Training DQN:  45%|████▌     | 901/2000 [17:11<35:31,  1.94s/it, Reward=-284, Epsilon=0.0109, Loss=8.77e+3, Fuel Usage=61.9] 
Episode 900, Average Reward: 2.45
Training DQN:  50%|█████     | 1001/2000 [20:12<30:24,  1.83s/it, Reward=-93.5, Epsilon=0.01, Loss=6.2e+3, Fuel Usage=61]    
Episode 1000, Average Reward: -6.84
Training DQN:  55%|█████▌    | 1101/2000 [22:58<20:55,  1.40s/it, Reward=209, Epsilon=0.01, Loss=2.4e+3, Fuel Usage=46.7]   
Episode 1100, Average Reward: 53.16
Training DQN:  60%|██████    | 1201/2000 [25:49<23:02,  1.73s/it, Reward=153, Epsilon=0.01, Loss=5.66e+3, Fuel Usage=59]    
Episode 1200, Average Reward: 54.31
Training DQN:  65%|██████▌   | 1301/2000 [28:06<14:09,  1.22s/it, Reward=274, Epsilon=0.01, Loss=2.31e+3, Fuel Usage=13.5]  
Episode 1300, Average Reward: 111.35
Training DQN:  70%|███████   | 1401/2000 [30:11<09:53,  1.01it/s, Reward=248, Epsilon=0.01, Loss=2.68e+3, Fuel Usage=16.1]  
Episode 1400, Average Reward: 156.67
Training DQN:  75%|███████▌  | 1501/2000 [32:06<09:58,  1.20s/it, Reward=306, Epsilon=0.01, Loss=2.57e+3, Fuel Usage=24.9]  
Episode 1500, Average Reward: 162.58
Training DQN:  80%|████████  | 1601/2000 [33:45<05:53,  1.13it/s, Reward=238, Epsilon=0.01, Loss=2.21e+3, Fuel Usage=32.5]  
Episode 1600, Average Reward: 198.48
Training DQN:  85%|████████▌ | 1701/2000 [35:30<03:22,  1.47it/s, Reward=246, Epsilon=0.01, Loss=2.62e+3, Fuel Usage=7.68]  
Episode 1700, Average Reward: 201.26
Training DQN:  90%|█████████ | 1801/2000 [36:59<02:58,  1.12it/s, Reward=254, Epsilon=0.01, Loss=4.39e+3, Fuel Usage=23.9]  
Episode 1800, Average Reward: 225.50
Training DQN:  95%|█████████▌| 1901/2000 [38:22<01:34,  1.05it/s, Reward=249, Epsilon=0.01, Loss=3.91e+3, Fuel Usage=8.25]  
Episode 1900, Average Reward: 212.99
Training DQN: 100%|██████████| 2000/2000 [40:06<00:00,  1.20s/it, Reward=249, Epsilon=0.01, Loss=2.73e+3, Fuel Usage=9.39]  

In [107]:
# win_env_dqn_train.plot_results()

In [108]:
# win_env_dqn_train.save_model("dqn_lunarlander-win-env-2k-adam.pth")

In [109]:
# win_env_dqn_train.test_model(episodes=500)

100%|██████████| 500/500 [00:31<00:00, 15.65it/s]
Success: 417/500 | Success Rate: 83.40%

In [110]:
# win_env_dqn_train.display_sample_video(sample_video=5)

In [120]:
Video("videos-dqn-wind/rl-video-episode-4.mp4")

**Nhận xét:**
Có thể thấy rằng khi hoạt động trong môi trường có sự nhiễu loạn (như gió và biến động môi trường), hiệu suất của DQN giảm rõ rệt. Cụ thể, mức average reward > 200 chỉ đạt được sau khoảng 1700 episodes, so với mốc 800 episodes trong môi trường tiêu chuẩn. Tỷ lệ thành công (success rate) cũng giảm, chỉ còn 83%, thay vì >90% như trước. Đồng thời, thời gian huấn luyện tăng đáng kể lên đến khoảng 40 phút — cho thấy mức độ khó khăn tăng lên khi môi trường trở nên bất định hơn.
Điều này có thể là do:
- DQN dễ overfit vào các mẫu trạng thái quen thuộc: DQN có xu hướng học "quá mức" vào các mẫu quen thuộc, không có khả năng thay đổi để ứng phó với môi trường mới.

## c. Tối ưu với DDQN

Nhận thấy sự thiếu sót của DQN trong môi trường có nhiễu loạn, dưới đây là triển khai của mô hình Double DQN (DDQN) cho hiệu suất tốt hơn ở môi trường nhiễu loạn.

In [111]:
class DDQNTrain:
    def __init__(self, env, optimizer, epsilon, min_epsilon, decay, gamma, batch_size, episodes, target_update_freq, memory_size, learning_rate):
        self.env = env
        self.episodes = episodes
        self.epsilon = epsilon
        self.min_epsilon = min_epsilon
        self.decay = decay
        self.gamma = gamma
        self.batch_size = batch_size
        self.target_update_freq = target_update_freq
        self.memory = deque(maxlen=memory_size)
        self.fuel_usage = []
        self.rewards = []
        self.losses = []

        self.q_net = DQNetwork(state_dim, action_dim).to(device)
        self.target_net = DQNetwork(state_dim, action_dim).to(device)
        self.target_net.load_state_dict(self.q_net.state_dict())
        self.target_net.eval()

        if optimizer == 'adam':
            self.optimizer = torch.optim.Adam(self.q_net.parameters(), lr=learning_rate)
        elif optimizer == 'rmsprop':
            self.optimizer = torch.optim.RMSprop(self.q_net.parameters(), lr=learning_rate)
        elif optimizer == 'sgd':
            self.optimizer = torch.optim.SGD(self.q_net.parameters(), lr=learning_rate)

    def select_action(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, action_dim - 1)
        else:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
                return self.q_net(state_tensor).argmax().item()

    def train_step(self):
        if len(self.memory) < self.batch_size:
            return 0
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(np.array(states)).to(device)
        actions = torch.LongTensor(actions).unsqueeze(1).to(device)
        rewards = torch.FloatTensor(rewards).unsqueeze(1).to(device)
        next_states = torch.FloatTensor(np.array(next_states)).to(device)
        dones = torch.FloatTensor(dones).unsqueeze(1).to(device)

        q_values = self.q_net(states).gather(1, actions)

        with torch.no_grad():
            next_actions = self.q_net(next_states).argmax(1).unsqueeze(1)
            next_q_values = self.target_net(next_states).gather(1, next_actions)
            targets = rewards + self.gamma * next_q_values * (1 - dones)

        loss = nn.MSELoss()(q_values, targets)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()

    def train(self):
        pbar = tqdm(range(self.episodes), desc="Training DDQN")
        for ep in pbar:
            state, _ = self.env.reset()
            total_reward = 0
            total_loss = 0
            total_fuel = 0
            done = False

            while not done:
                action = self.select_action(state)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                self.memory.append((state, action, reward, next_state, float(done)))

                loss = self.train_step()
                total_loss += loss
                total_reward += reward
                total_fuel += fuel_cost[action]

                state = next_state

            if ep % self.target_update_freq == 0:
                self.target_net.load_state_dict(self.q_net.state_dict())

            if ep % 100 == 0:
                average_reward = np.mean(self.rewards[-100:])
                print(f"Episode {ep}, Average Reward: {average_reward:.2f}")

            self.epsilon = max(self.min_epsilon, self.epsilon * self.decay)
            self.rewards.append(total_reward)
            self.losses.append(total_loss)
            self.fuel_usage.append(total_fuel)

            pbar.set_postfix({
                "Reward": total_reward,
                "Epsilon": self.epsilon,
                "Loss": total_loss,
                "Fuel Usage": total_fuel
            })

        return self.rewards, self.losses, self.fuel_usage

    def save_model(self, path):
        '''
        - Lưu trọng số của mạng Q vào tệp
        '''
        torch.save(self.target_net.state_dict(), path)


    def load_model(self, path):
        '''
        - Tải trọng số của mạng Q từ tệp
        - Đặt mạng Q mục tiêu ở chế độ đánh giá để không cập nhật trọng số trong quá trình kiểm tra
        '''
        self.target_net.load_state_dict(torch.load(path))
        self.target_net.eval()
        
        
    def get_target_action(self, state):
        ''' 
        - Chọn hành động từ mạng Q mục tiêu để kiểm tra
        '''
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            return self.target_net(state_tensor).argmax().item()


    def plot_results(self):
        '''
        - Vẽ biểu đồ phần thưởng, tổn thất và mức tiêu thụ nhiên liệu
        - Sử dụng các hàm plot_rewards, plot_losses và plot_fuel_usage đã định nghĩa ở trên
        '''
        plot_rewards(self.rewards)
        plot_losses(self.losses)
        plot_fuel_usage(self.fuel_usage)
        
    
    def test_model(self, episodes=500):
        '''
        - Kiểm tra mô hình đã được huấn luyện bằng cách chạy một số tập
        - Trong mỗi tập, khởi tạo trạng thái và phần thưởng
        - Lặp qua các bước trong mỗi tập cho đến khi hoàn thành
        - Chọn hành động bằng cách sử dụng hàm get_target_action
        - Thực hiện hành động và nhận trạng thái tiếp theo, phần thưởng và trạng thái hoàn thành
        - Tính toán tổng phần thưởng
        - Nếu tổng phần thưởng lớn hơn hoặc bằng 200, tăng biến thành công
        - Trả về số lượng thành công và tỷ lệ thành công
        '''
        total_success = 0
        testing = gym.make("LunarLander-v3")
        
        for episode in tqdm(range(episodes)):
            state, _ = testing.reset()
            done = False
            total_reward = 0
            
            while not done:
                action = self.get_target_action(state)
                state, reward, terminated, truncated, _ = testing.step(action)
                total_reward += reward
                done = terminated or truncated
            if total_reward >= 200:
                total_success += 1
        print(f"Success: {total_success}/{episodes} | Success Rate: {total_success / episodes * 100:.2f}%")
        testing.close()
        
        
    def display_sample_video(self, sample_video=1):
        '''
        - Hiển thị video mẫu của mô hình đã được huấn luyện
        - Sử dụng RecordVideo để ghi lại video trong môi trường
        - Trong mỗi tập, khởi tạo trạng thái và phần thưởng
        - Lặp qua các bước trong mỗi tập cho đến khi hoàn thành
        - Chọn hành động bằng cách sử dụng hàm get_target_action
        - Thực hiện hành động và nhận trạng thái tiếp theo, phần thưởng và trạng thái hoàn thành
        - Đóng video sau khi hoàn thành
        - Trả về video cuối cùng được ghi lại
        '''
        video_render = gym.make("LunarLander-v3", render_mode="rgb_array")
        video_render = RecordVideo(video_render, "videos", episode_trigger=lambda x: True)
        
        for _ in tqdm(range(sample_video)):
            state, _ = video_render.reset()
            done = False
            while not done:
                action = self.get_target_action(state)
                state, reward, terminated, truncated, _ = video_render.step(action)
                done = terminated or truncated
            video_render.close()
        video_files = glob.glob("videos/*.mp4")
        return Video(video_files[-1])

In [112]:
ddqn_trainer = DDQNTrain(
    env=win_env,
    optimizer='adam',
    epsilon=1.0,
    min_epsilon=0.01,
    decay=0.995,
    gamma=0.99,
    batch_size=128,
    episodes=2000,
    target_update_freq=10,
    memory_size=100000,
    learning_rate=1e-4
)

In [113]:
# rewards, losses, fuel_usage = ddqn_trainer.train()

Training DDQN:   0%|          | 0/2000 [00:00<?, ?it/s, Reward=-211, Epsilon=0.995, Loss=0, Fuel Usage=9.48]
Episode 0, Average Reward: nan
Training DDQN:   5%|▌         | 101/2000 [00:46<16:14,  1.95it/s, Reward=-191, Epsilon=0.603, Loss=2.23e+3, Fuel Usage=6.36]
Episode 100, Average Reward: -221.90
Training DDQN:  10%|█         | 201/2000 [01:38<14:58,  2.00it/s, Reward=-110, Epsilon=0.365, Loss=1.8e+3, Fuel Usage=11.2]  
Episode 200, Average Reward: -138.13
Training DDQN:  15%|█▌        | 301/2000 [03:26<47:23,  1.67s/it, Reward=-258, Epsilon=0.221, Loss=1.86e+3, Fuel Usage=11.8]   
Episode 300, Average Reward: -131.86
Training DDQN:  20%|██        | 401/2000 [05:35<23:06,  1.15it/s, Reward=2.19, Epsilon=0.134, Loss=1.84e+3, Fuel Usage=4.26]  
Episode 400, Average Reward: -109.48
Training DDQN:  25%|██▌       | 501/2000 [08:25<33:51,  1.36s/it, Reward=-115, Epsilon=0.0812, Loss=2.16e+3, Fuel Usage=10.9]   
Episode 500, Average Reward: -106.71
Training DDQN:  30%|███       | 601/2000 [11:07<37:20,  1.60s/it, Reward=-141, Epsilon=0.0492, Loss=1.31e+3, Fuel Usage=6.81]  
Episode 600, Average Reward: -82.40
Training DDQN:  35%|███▌      | 701/2000 [14:19<37:56,  1.75s/it, Reward=-125, Epsilon=0.0298, Loss=1.34e+4, Fuel Usage=76.7]   
Episode 700, Average Reward: 18.65
Training DDQN:  40%|████      | 801/2000 [16:43<23:58,  1.20s/it, Reward=236, Epsilon=0.018, Loss=3.97e+3, Fuel Usage=19.6]   
Episode 800, Average Reward: -81.40
Training DDQN:  45%|████▌     | 901/2000 [18:55<30:23,  1.66s/it, Reward=-217, Epsilon=0.0109, Loss=7.54e+3, Fuel Usage=37.8] 
Episode 900, Average Reward: -52.22
Training DDQN:  50%|█████     | 1001/2000 [21:01<16:14,  1.02it/s, Reward=-11.7, Epsilon=0.01, Loss=2.17e+3, Fuel Usage=19.8] 
Episode 1000, Average Reward: -11.73
Training DDQN:  55%|█████▌    | 1101/2000 [23:06<22:50,  1.52s/it, Reward=103, Epsilon=0.01, Loss=7.52e+3, Fuel Usage=38.7]  
Episode 1100, Average Reward: 113.59
Training DDQN:  60%|██████    | 1201/2000 [25:05<13:59,  1.05s/it, Reward=235, Epsilon=0.01, Loss=4.84e+3, Fuel Usage=23.2]  
Episode 1200, Average Reward: 158.15
Training DDQN:  65%|██████▌   | 1301/2000 [27:03<13:36,  1.17s/it, Reward=240, Epsilon=0.01, Loss=4.75e+3, Fuel Usage=30.1]  
Episode 1300, Average Reward: 192.16
Training DDQN:  70%|███████   | 1401/2000 [29:00<09:37,  1.04it/s, Reward=235, Epsilon=0.01, Loss=3.23e+3, Fuel Usage=11.8]  
Episode 1400, Average Reward: 216.21
Training DDQN:  75%|███████▌  | 1501/2000 [30:56<08:47,  1.06s/it, Reward=229, Epsilon=0.01, Loss=3.44e+3, Fuel Usage=21.5]  
Episode 1500, Average Reward: 211.82
Training DDQN:  80%|████████  | 1601/2000 [32:56<06:04,  1.09it/s, Reward=205, Epsilon=0.01, Loss=2.14e+3, Fuel Usage=15.1]  
Episode 1600, Average Reward: 205.14
Training DDQN:  85%|████████▌ | 1701/2000 [34:37<04:03,  1.23it/s, Reward=265, Epsilon=0.01, Loss=2.46e+3, Fuel Usage=19.8]  
Episode 1700, Average Reward: 213.45
Training DDQN:  90%|█████████ | 1801/2000 [36:08<02:50,  1.17it/s, Reward=267, Epsilon=0.01, Loss=3.65e+3, Fuel Usage=26.2]  
Episode 1800, Average Reward: 193.01
Training DDQN:  95%|█████████▌| 1901/2000 [37:37<01:26,  1.15it/s, Reward=246, Epsilon=0.01, Loss=5.43e+3, Fuel Usage=21.8]  
Episode 1900, Average Reward: 167.89
Training DDQN: 100%|██████████| 2000/2000 [38:56<00:00,  1.17s/it, Reward=250, Epsilon=0.01, Loss=4.83e+3, Fuel Usage=16.8]  

In [114]:
# ddqn_trainer.plot_results()

In [115]:
# ddqn_trainer.save_model("ddqn_lunarlander-win-env-2k-adam.pth")

In [116]:
# ddqn_trainer.load_model("ddqn_lunarlander-win-env-2k-adam.pth")

In [117]:
# ddqn_trainer.test_model(episodes=500)

100%|██████████| 500/500 [00:22<00:00, 21.95it/s]
Success: 471/500 | Success Rate: 94.20%

In [118]:
# ddqn_trainer.display_sample_video(sample_video=5)

In [119]:
Video("videos-ddqn-wind/rl-video-episode-4.mp4")

Có thể thấy DDQN có tốc độ học nhỉnh hơn so với DQN (38p và 40p) trong khi độ chính xác tăng cao
- Đạt AVG-Reward > 200 tại ep1400 trong khi DQN đạt > 200 tại ep1700
- Đạt success rate 93% (có thể hiệu suất bị giảm trong 200ep cuối do mô hình chưa ổn định, nếu train lâu hơn có thể đạt độ chính xác cao hơn)

Lý do: 
- Giảm overestimation nhờ tách biệt quá trình chọn và đánh giá hành động.
- Hội tụ ổn định hơn trong môi trường nhiễu.
- Chính sách học được tổng quát tốt hơn, giúp tăng độ chính xác và hiệu suất.